Ref: https://pytorch.org/docs/master/onnx.html#example-end-to-end-alexnet-from-pytorch-to-onnx

In [2]:
import io
import numpy as np
import torch
import torch.onnx
import torch.nn as nn
import torchvision

In [6]:
model = torchvision.models.alexnet(pretrained=False)
alexnet_weights = r"D:\CETCA_DeepLearning\yolo_cfg_weights\alexnet-owt-4df8aa71.pth"
model.load_state_dict(torch.load(alexnet_weights))
model.cuda()
model.train(False)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inplace)
    (3): Dropout(p

导出onnx模型时，需要做一次推理过程用于记录推理流程路径，所有需要给定一个模型输入数据。<br>
官方解释为：This exporter runs your model once in order to get a trace of its execution to be exported; at the moment, it supports a limited set of dynamic models (e.g., RNNs.)

In [7]:
file_out_model = r"D:\CETCA_DeepLearning\yolo_cfg_weights\alexnet.onnx"
dummy_input = torch.randn(10,3,224,224,device='cuda')
dummy_output = model(dummy_input)
torch.onnx.export(model,dummy_input,file_out_model,verbose=True)

graph(%0 : Float(10, 3, 224, 224),
      %features.0.weight : Float(64, 3, 11, 11),
      %features.0.bias : Float(64),
      %features.3.weight : Float(192, 64, 5, 5),
      %features.3.bias : Float(192),
      %features.6.weight : Float(384, 192, 3, 3),
      %features.6.bias : Float(384),
      %features.8.weight : Float(256, 384, 3, 3),
      %features.8.bias : Float(256),
      %features.10.weight : Float(256, 256, 3, 3),
      %features.10.bias : Float(256),
      %classifier.1.weight : Float(4096, 9216),
      %classifier.1.bias : Float(4096),
      %classifier.4.weight : Float(4096, 4096),
      %classifier.4.bias : Float(4096),
      %classifier.6.weight : Float(1000, 4096),
      %classifier.6.bias : Float(1000)):
  %17 : Float(10, 64, 55, 55) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[11, 11], pads=[2, 2, 2, 2], strides=[4, 4]](%0, %features.0.weight, %features.0.bias), scope: AlexNet/Sequential[features]/Conv2d[0]
  %18 : Float(10, 64, 55, 55) = onnx::Relu(%17), 